In [43]:
import numpy as np
import itertools

X, Y = [], []

ts = [2,3,84,5,1,54,2,45,34,78]
lag = 3

if len(ts) - lag <= 0:
    X.append(ts)
else:
    for i in range(len(ts) - lag):
        Y.append(ts[i + lag])
        # Substacted 96 for not knowing the day before
        #ab = list(itertools.chain(ts[i:(i + lag)], [2,3]))
        #X.append(ab)
        X.append(ts[i:(i + lag)])
    
#X = list(X)
X, Y = np.array(X), np.array(Y)

# Reshaping the X array to an LSTM input shape 
X = np.reshape(X, (1, X.shape[0], X.shape[1]))

In [44]:
print(X.shape)

(1, 7, 3)


In [42]:
print(X)

[[ 2  3 84]
 [ 3 84  5]
 [84  5  1]
 [ 5  1 54]
 [ 1 54  2]
 [54  2 45]
 [ 2 45 34]]
